In [10]:
import datetime
import numpy as np
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt
import pandas as pd

In [7]:
with PdfPages('multipage_pdf.pdf') as pdf:
    #plt.figure(1)
    plt.plot(range(7), [3, 1, 4, 1, 5, 9, 2], 'r-o')
    plt.title('id = 1')
    pdf.savefig()  # saves the current figure into a pdf page
    plt.close()

    #plt.figure(2)
    plt.plot(range(7), [4, 2, 3, 4, 5, 6, 1], 'r-o')
    plt.title('id = 2')
    pdf.savefig()  # saves the current figure into a pdf page
    plt.close()


In [12]:
li = [1,2,3,1,2,4,5,6,3]
dic = {}
for i in li:
    if i in dic:
        dic[i] += 1
    else:
        dic[i] = 1
print(dic)
data = pd.DataFrame({'type':li})
print(data)

{1: 2, 2: 2, 3: 2, 4: 1, 5: 1, 6: 1}
   type
0     1
1     2
2     3
3     1
4     2
5     4
6     5
7     6
8     3
